In [1]:
import pandas as pd
import pymysql
from pyhive import hive
import pandas as pd
from pyspark import SparkContext,SQLContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
import datetime


In [2]:
spark=SparkSession \
        .builder \
        .config("spark.eventLog.enabled", "false") \
        .config("spark.executor.memory", "4g")\
        .config("spark.driver.memory", "8g")\
        .config("spark.cores.max", "10")\
        .config("spark.task.maxFailures", "1000")\
        .config("spark.default.parallelism", "500")\
        .config("spark.sql.shuffle.partitions",50)\
        .appName('renhang_etl') \
        .master('yarn')\
        .getOrCreate()

In [3]:
class hiveParse(object):
    """
    该方法用于读取hive 数据，并转化成pandas dataframe
    """
    def __init__(self, host='', port=10000, user='', passwd='', dbName=''):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName

    def conn_hive(self):
        self.connection = hive.Connection(host=self.host,port=self.port,username=self.user)

    def hive_connect(self,sql_select):
        cur = self.connection.cursor()
        cur.execute(sql_select)
        columns = [col[0] for col in cur.description]
        data = cur.fetchall()
        query_df = pd.DataFrame(data)
        query_df.columns = columns
        return query_df

class mysqldbParse(object):
    """
    该方法用于连接mysql，并执行相应操作
    """

    def __init__(self, host='', port=3306, user='', passwd='', dbName='', charset='utf8', connect_timeout=31536000):
        self.host = host
        self.port = port
        self.user = user
        self.passwd = passwd
        self.dbName = dbName
        self.charset = charset
        self.connect_timeout = connect_timeout

    def conn_mysql(self):
        self.conn = pymysql.connect(host=self.host,
                                    port=self.port,
                                    user=self.user,
                                    password=self.passwd,
                                    db=self.dbName,
                                    charset=self.charset,
                                    connect_timeout=self.connect_timeout,
                                    cursorclass=pymysql.cursors.DictCursor)

    def insert_sql(self, insertSql):
        with self.conn.cursor() as cursor:
            cursor.execute(insertSql)
            self.conn.commit()

    def select_Sql(self, selectSql):
        df = pd.read_sql(selectSql, self.conn)
        return df

    def close(self):
        self.conn.close()

In [4]:
def hive_df(sql):
    host = "58.59.18.61"
    port = 10000
    user = "hdfs"
    dbName = "ods"
    hive_parse = hiveParse(host,port,user,dbName)
    hive_parse.conn_hive()
    hive_pd = hive_parse.hive_connect(sql)
    return hive_pd

def mysql_df(selectSql):
    host = "58.59.11.86"
    port = 3306
    user = "tangdinghai"
    passwd = "Tangdinghai2019!"
    dbName = "reportpublic"
    mysql_parse = mysqldbParse(host,port,user,passwd,dbName)
    mysql_parse.conn_mysql()
    mysql_pd = mysql_parse.select_Sql(selectSql)
    mysql_parse.close()
    return mysql_pd



In [68]:
#### 头条分发模型：有授信额度相关客户级和订单级汇总监控报表---日报

In [5]:
base_dict = {'risk_level':['A','B','C','D','E','F','汇总'],\
             'toutiao_develop':['12.99%','17.12%','18.46%','20.43%','16.88%','14.12%','14303']}
dairy_report = pd.DataFrame(base_dict)

In [6]:
table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_credit_credit_info") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_credit_credit_info")

table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_credit_credit_apply") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_credit_credit_apply")

table1 = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:mysql://58.59.11.86:3306") \
    .option("dbtable", "reportpublic.ods_loan_invoice") \
    .option("user", "tangdinghai") \
    .option("password", "Tangdinghai2019!") \
    .load()
table1.createOrReplaceTempView("ods_loan_invoice")

In [7]:
hive_sql = """


select
	risk_level,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when t.OUT_FRONT_RISK_DEGREE  = 'A' then "A"
			when t.OUT_FRONT_RISK_DEGREE  = 'B'  then "B"
			when t.OUT_FRONT_RISK_DEGREE  = 'C'  then "C"
			when t.OUT_FRONT_RISK_DEGREE  = 'D'  then "D"
			when t.OUT_FRONT_RISK_DEGREE  = 'E'  then "E"
			when t.OUT_FRONT_RISK_DEGREE  = 'F'  then "F"
		end as risk_level
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score,
                    b.OUT_FRONT_RISK_DEGREE as OUT_FRONT_RISK_DEGREE
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

                left JOIN
                (
                    SELECT 
                        session_id,
                        user_id,
                        out_credit_score,
                        OUT_FRONT_RISK_DEGREE
                    FROM ods.ods_toutiao_shouxin_output
                ) AS b ON a.session_id = b.session_id
                

                
        ) as t  where frixtimom_une(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s  
) as a group by a.risk_level



"""
# toutiao = spark.sql(hive_sql).toPandas()

In [8]:
def dairly_monitor(sql,flag,daydiff1=0,daydiff2=0):
    
    if(flag == 'today'):
        time1 = """date_add(CURRENT_DATE,-1)"""
        time2 = """date_add(CURRENT_DATE,-1)"""
        flag = 'today'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    if(flag == 'week'):
        time1 = """date_add(CURRENT_DATE,-%d)"""%daydiff1
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
        
    if(flag == 'month'):
        time1 = """date_add(add_months(CURRENT_DATE, -%d ),-%d)"""%(daydiff1,daydiff2)
        time2 = """date_add(CURRENT_DATE,-%d)"""%daydiff2
        excute_sql = sql%(flag,flag,time1,time2)
    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

def monthly_monitor(sql,flag,monthdiff):
    
    if(flag == 'online'):
        time1 = """date_add(last_day(add_months('2019-01-14', -1 )),1)"""
        time2 = """last_day('2019-01-14')"""
        flag = 'online'
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)
    else:
        time1 = """date_add(last_day(add_months(CURRENT_DATE, -%d )),1)"""%(monthdiff+1)    #当月第一天：上月最后一天+1天
        time2 = """last_day(add_months(CURRENT_DATE,-%d))"""%monthdiff                  #当月最后一天
        
        date_sql = """select last_day(add_months(CURRENT_DATE,-%d)) as dt"""%monthdiff
        date =  hive_df(date_sql)['dt'].astype('str')
        flag = date[0][0:7].replace('-','_')
        excute_sql = sql%(flag,flag,time1,time2)
#         print(excute_sql)

    #获取每个月的时间
    credit_apply = spark.sql(excute_sql).toPandas()
    return credit_apply

In [12]:
dairly_apply_day = dairly_monitor(hive_sql,'today',0,0)
dairly_apply_week = dairly_monitor(hive_sql,'week',8,2)
dairly_apply_month = dairly_monitor(hive_sql,'month',1,2)


join_key = 'risk_level'

apply_monitor_dairly = dairy_report.merge(dairly_apply_month,on = join_key ,how='left')\
                                  .merge(dairly_apply_week,on = join_key, how='left')\
                                  .merge(dairly_apply_day,on = join_key, how='left').fillna(0)

col_keep = list(apply_monitor_dairly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_dairly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    apply_monitor_dairly[col][idx] = col_total
    apply_monitor_dairly[col+"_percent"] = apply_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_dairly[col+"_percent"][idx] = col_total 

    
apply_monitor_dairly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,risk_level,toutiao_develop,apply_uid_month,apply_sessionId_month,apply_uid_week,apply_sessionId_week,apply_uid_today,apply_sessionId_today,apply_uid_month_percent,apply_sessionId_month_percent,apply_uid_week_percent,apply_sessionId_week_percent,apply_uid_today_percent,apply_sessionId_today_percent
0,A,12.99%,137.0,137.0,81.0,81.0,0.0,0.0,0.092881,0.092818,0.077364,0.077290,0.0,0.0
1,B,17.12%,39.0,39.0,27.0,27.0,0.0,0.0,0.026441,0.026423,0.025788,0.025763,0.0,0.0
2,C,18.46%,574.0,574.0,424.0,424.0,0.0,0.0,0.389153,0.388889,0.404967,0.404580,0.0,0.0
3,D,20.43%,235.0,235.0,166.0,166.0,0.0,0.0,0.159322,0.159214,0.158548,0.158397,0.0,0.0
4,E,16.88%,404.0,405.0,281.0,282.0,0.0,0.0,0.273898,0.274390,0.268386,0.269084,0.0,0.0
5,F,14.12%,86.0,86.0,68.0,68.0,0.0,0.0,0.058305,0.058266,0.064947,0.064885,0.0,0.0
6,汇总,14303,1475.0,1476.0,1047.0,1048.0,0.0,0.0,1475.000000,1476.000000,1047.000000,1048.000000,0.0,0.0


In [13]:
#上线月
apply_month_online = monthly_monitor(hive_sql,'online',-1)
#当月
apply_month_now = monthly_monitor(hive_sql,'not_online',0)
#上月
apply_month_last1 = monthly_monitor(hive_sql,'not_online',1)
#上两月
apply_month_last2 = monthly_monitor(hive_sql,'not_online',2)
#上3月
apply_month_last3 = monthly_monitor(hive_sql,'not_online',3)
#上4月
apply_month_last4 = monthly_monitor(hive_sql,'not_online',4)
#上5月
apply_month_last5 = monthly_monitor(hive_sql,'not_online',5)
#上6月
apply_month_last6 = monthly_monitor(hive_sql,'not_online',6)



join_key = 'risk_level'

apply_monitor_monthly = dairy_report.merge(apply_month_online,on = join_key,how='left')\
                                  .merge(apply_month_last6,on = join_key,how='left')\
                                  .merge(apply_month_last5,on = join_key,how='left')\
                                  .merge(apply_month_last4,on = join_key,how='left')\
                                  .merge(apply_month_last3,on = join_key,how='left')\
                                  .merge(apply_month_last2,on = join_key,how='left')\
                                  .merge(apply_month_last1,on = join_key,how='left')\
                                  .merge(apply_month_now,on = join_key,how='left').fillna(0)

col_keep = list(apply_monitor_monthly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(apply_monitor_monthly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    apply_monitor_monthly[col][idx] = col_total
    apply_monitor_monthly[col+"_percent"] = apply_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    apply_monitor_monthly[col+"_percent"][idx] = col_total 

    
apply_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,risk_level,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,A,12.99%,137.0,137.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092881,0.092818
1,B,17.12%,39.0,39.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026441,0.026423
2,C,18.46%,574.0,574.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389153,0.388889
3,D,20.43%,235.0,235.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159322,0.159214
4,E,16.88%,404.0,405.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.273898,0.274390
5,F,14.12%,86.0,86.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058305,0.058266
6,汇总,14303,1475.0,1476.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1475.000000,1476.000000


In [14]:
##### 授信相关日报和月报

In [15]:
shouxin_hive_sql = """


select
	risk_level,
	count(distinct(user_id)) as apply_uid_%s,
    count(distinct(session_id)) as apply_sessionId_%s

from
(
	select
		user_id,
        session_id,
		case
			when t.OUT_FRONT_RISK_DEGREE  = 'A' then "A"
			when t.OUT_FRONT_RISK_DEGREE  = 'B'  then "B"
			when t.OUT_FRONT_RISK_DEGREE  = 'C'  then "C"
			when t.OUT_FRONT_RISK_DEGREE  = 'D'  then "D"
			when t.OUT_FRONT_RISK_DEGREE  = 'E'  then "E"
			when t.OUT_FRONT_RISK_DEGREE  = 'F'  then "F"
		end as risk_level
	from 
			(
				SELECT
     				a.session_id as session_id,
     				a.user_id as user_id,
     				a.t02_apply_time as apply_time,
     				b.out_credit_score as out_credit_score,
                    b.OUT_FRONT_RISK_DEGREE as OUT_FRONT_RISK_DEGREE
				FROM 
				(
					SELECT 
						session_id,
       					user_id,
       					t02_apply_time
					FROM ods.ods_toutiao_shouxin_input 
				) AS a

                left JOIN
                (
                    SELECT 
                        session_id,
                        user_id,
                        out_credit_score,
                        OUT_FRONT_RISK_DEGREE
                    FROM ods.ods_toutiao_shouxin_output
                ) AS b ON a.session_id = b.session_id
                
                right join 
                (
                
                    select 
                        user_id
                    from ods_credit_credit_info
                
                ) as c on a.user_id = c.user_id
                
        ) as t  where from_unixtime(unix_timestamp(substr(t.apply_time,0,8),'yyyymmdd'),'yyyy-mm-dd') 
            between %s and %s   
) as a group by a.risk_level



"""

# dairly_shouxin_day = dairly_monitor(shouxin_hive_sql,'today',0,0)


In [16]:
dairly_shouxin_day = dairly_monitor(shouxin_hive_sql,'today',0,0)
dairly_shouxin_week = dairly_monitor(shouxin_hive_sql,'week',8,2)
dairly_shouxin_month = dairly_monitor(shouxin_hive_sql,'month',1,2)


join_key = 'risk_level'

shouxin_monitor_dairly = dairy_report.merge(dairly_shouxin_month,on = join_key ,how='left')\
                                  .merge(dairly_shouxin_week,on = join_key, how='left')\
                                  .merge(dairly_shouxin_day,on = join_key, how='left').fillna(0)

col_keep = list(shouxin_monitor_dairly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_dairly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    shouxin_monitor_dairly[col][idx] = col_total
    shouxin_monitor_dairly[col+"_percent"] = shouxin_monitor_dairly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_dairly[col+"_percent"][idx] = col_total 

    
shouxin_monitor_dairly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,risk_level,toutiao_develop,apply_uid_month,apply_sessionId_month,apply_uid_week,apply_sessionId_week,apply_uid_today,apply_sessionId_today,apply_uid_month_percent,apply_sessionId_month_percent,apply_uid_week_percent,apply_sessionId_week_percent,apply_uid_today_percent,apply_sessionId_today_percent
0,A,12.99%,22.0,22.0,15.0,15.0,0.0,0.0,0.101382,0.101382,0.092593,0.092593,0.0,0.0
1,B,17.12%,7.0,7.0,4.0,4.0,0.0,0.0,0.032258,0.032258,0.024691,0.024691,0.0,0.0
2,C,18.46%,87.0,87.0,66.0,66.0,0.0,0.0,0.400922,0.400922,0.407407,0.407407,0.0,0.0
3,D,20.43%,36.0,36.0,25.0,25.0,0.0,0.0,0.165899,0.165899,0.154321,0.154321,0.0,0.0
4,E,16.88%,59.0,59.0,46.0,46.0,0.0,0.0,0.271889,0.271889,0.283951,0.283951,0.0,0.0
5,F,14.12%,6.0,6.0,6.0,6.0,0.0,0.0,0.027650,0.027650,0.037037,0.037037,0.0,0.0
6,汇总,14303,217.0,217.0,162.0,162.0,0.0,0.0,217.000000,217.000000,162.000000,162.000000,0.0,0.0


In [17]:
#上线月
shouxin_month_online = monthly_monitor(hive_sql,'online',-1)
#当月
shouxin_month_now = monthly_monitor(hive_sql,'not_online',0)
#上月
shouxin_month_last1 = monthly_monitor(hive_sql,'not_online',1)
#上两月
shouxin_month_last2 = monthly_monitor(hive_sql,'not_online',2)
#上3月
shouxin_month_last3 = monthly_monitor(hive_sql,'not_online',3)
#上4月
shouxin_month_last4 = monthly_monitor(hive_sql,'not_online',4)
#上5月
shouxin_month_last5 = monthly_monitor(hive_sql,'not_online',5)
#上6月
shouxin_month_last6 = monthly_monitor(hive_sql,'not_online',6)



join_key = 'risk_level'

shouxin_monitor_monthly = dairy_report.merge(shouxin_month_online,on = join_key,how='left')\
                                  .merge(shouxin_month_last6,on = join_key,how='left')\
                                  .merge(shouxin_month_last5,on = join_key,how='left')\
                                  .merge(shouxin_month_last4,on = join_key,how='left')\
                                  .merge(shouxin_month_last3,on = join_key,how='left')\
                                  .merge(shouxin_month_last2,on = join_key,how='left')\
                                  .merge(shouxin_month_last1,on = join_key,how='left')\
                                  .merge(shouxin_month_now,on = join_key,how='left').fillna(0)

col_keep = list(shouxin_monitor_monthly.columns)
col_keep.remove(join_key)
col_keep.remove('toutiao_develop')

for col in col_keep:
    col_total = sum(shouxin_monitor_monthly[col])
    idx = len(dairy_report['toutiao_develop']) - 1
    shouxin_monitor_monthly[col][idx] = col_total
    shouxin_monitor_monthly[col+"_percent"] = shouxin_monitor_monthly[col].map(lambda x: x/(col_total+0.000001))
    shouxin_monitor_monthly[col+"_percent"][idx] = col_total 

    
shouxin_monitor_monthly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,risk_level,toutiao_develop,apply_uid_online,apply_sessionId_online,apply_uid_2018_07,apply_sessionId_2018_07,apply_uid_2018_08,apply_sessionId_2018_08,apply_uid_2018_09,apply_sessionId_2018_09,...,apply_uid_2018_09_percent,apply_sessionId_2018_09_percent,apply_uid_2018_10_percent,apply_sessionId_2018_10_percent,apply_uid_2018_11_percent,apply_sessionId_2018_11_percent,apply_uid_2018_12_percent,apply_sessionId_2018_12_percent,apply_uid_2019_01_percent,apply_sessionId_2019_01_percent
0,A,12.99%,137.0,137.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.092881,0.092818
1,B,17.12%,39.0,39.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.026441,0.026423
2,C,18.46%,574.0,574.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.389153,0.388889
3,D,20.43%,235.0,235.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.159322,0.159214
4,E,16.88%,404.0,405.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.273898,0.274390
5,F,14.12%,86.0,86.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.058305,0.058266
6,汇总,14303,1475.0,1476.0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1475.000000,1476.000000


In [288]:
###########################################
#### 有授信额度相关客户级和订单级汇总监控报表---月报

In [58]:
#### 申请表通过率

In [18]:
accept_rate_dairly = shouxin_monitor_dairly[['risk_level']]
idx = len(dairy_report['toutiao_develop']) - 1

for i in range(len(shouxin_monitor_dairly.columns)):
    if(i>=2 and i <=7):
        shouxin_col = shouxin_monitor_dairly.columns[i]
        apply_col = apply_monitor_dairly.columns[i]
#         print(shouxin_col,apply_col)

        accept_col = shouxin_col.split("_")[1]+"_"+shouxin_col.split("_")[2]
        accept_rate_dairly[accept_col+"_rate"] = shouxin_monitor_dairly[shouxin_col]/(apply_monitor_dairly[apply_col])
        accept_rate_dairly[accept_col+"_rate"][idx] = apply_monitor_dairly[apply_col][idx]

accept_rate_dairly

/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:3267: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://p

,risk_level,uid_month_rate,sessionId_month_rate,uid_week_rate,sessionId_week_rate,uid_today_rate,sessionId_today_rate
0,A,0.160584,0.160584,0.185185,0.185185,NaN,NaN
1,B,0.179487,0.179487,0.148148,0.148148,NaN,NaN
2,C,0.151568,0.151568,0.155660,0.155660,NaN,NaN
3,D,0.153191,0.153191,0.150602,0.150602,NaN,NaN
4,E,0.146040,0.145679,0.163701,0.163121,NaN,NaN
5,F,0.069767,0.069767,0.088235,0.088235,NaN,NaN
6,汇总,1475.000000,1476.000000,1047.000000,1048.000000,0.0,0.0


In [20]:
accept_rate_monthly = shouxin_monitor_monthly[['risk_level']]
idx = len(dairy_report['toutiao_develop']) - 1

for i in range(len(shouxin_monitor_monthly.columns)):
    if(i>=2 and i <=16):
        shouxin_col = shouxin_monitor_monthly.columns[i]
        apply_col = apply_monitor_monthly.columns[i]
#       print(shouxin_col,apply_col)

        accept_col = shouxin_col[6:]
        accept_rate_monthly[accept_col+"_rate"] = shouxin_monitor_monthly[shouxin_col]/(apply_monitor_monthly[apply_col])
        accept_rate_monthly[accept_col+"_rate"][idx] = apply_monitor_monthly[apply_col][idx]

accept_rate_monthly = accept_rate_monthly.fillna("0")
accept_rate_monthly


/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()
/root/.pyenv/versions/3.5.1/envs/vpy3.5/lib/python3.5/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':


,risk_level,uid_online_rate,sessionId_online_rate,uid_2018_07_rate,sessionId_2018_07_rate,uid_2018_08_rate,sessionId_2018_08_rate,uid_2018_09_rate,sessionId_2018_09_rate,uid_2018_10_rate,sessionId_2018_10_rate,uid_2018_11_rate,sessionId_2018_11_rate,uid_2018_12_rate,sessionId_2018_12_rate,uid_2019_01_rate
0,A,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
1,B,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
2,C,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
3,D,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
4,E,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
5,F,1.0,1.0,0,0,0,0,0,0,0,0,0,0,0,0,1.0
6,汇总,1475.0,1476.0,0,0,0,0,0,0,0,0,0,0,0,0,1475.0
